# TEAL and smart contract on algorand

### Open cmd


######   At this point we are ready to start the node. Type the following command and press enter.
`
goal node start
`

######   You can verify the node is running by waiting for a minute and executing this
`
goal node status
`

######   to stop the node. Type the following command and press enter.
`
goal node stop
`

# Useful Links


 
## [Understand what is smart contract](https://www.youtube.com/watch?v=Fw9L54BvD2U "https://www.youtube.com/watch?v=Fw9L54BvD2U")

## [Developer Office Hours | Algorand’s Smart Contract Language](https://www.youtube.com/watch?v=HKJcc2nmjPo "https://www.youtube.com/watch?v=HKJcc2nmjPo")

## [Understanding Algorand Smart Contracts](https://medium.com/algorand/understanding-algorand-smart-contracts-b9fc743e7a0f "https://medium.com/algorand/understanding-algorand-smart-contracts-b9fc743e7a0f")



## [TEAL templates](https://github.com/algorand/go-algorand/tree/master/tools/teal/templates "https://github.com/algorand/go-algorand/tree/master/tools/teal/templates")


## [PyTeal — Writing Algorand Smart Contracts in Python](https://medium.com/algorand/pyteal-writing-algorand-smart-contracts-in-python-acfd7f7a48dd "https://medium.com/algorand/pyteal-writing-algorand-smart-contracts-in-python-acfd7f7a48dd")


In [39]:

# #!/usr/bin/env python3

# from pyteal import *

# #template variables
# tmpl_fee = Int(1000)
# tmpl_period = Int(1000)
# tmpl_dur = Int(1000)
# tmpl_x = Bytes("base64", "y9OJ5MRLCHQj8GqbikAUKMBI7hom+SOj8dlopNdNHXI=")
# tmpl_amt = Int(2000)
# tmpl_rcv = Addr("ZZAF5ARA4MEC5PVDOP64JM5O5MQST63Q2KOY2FLYFLXXD3PFSNJJBYAFZM")
# tmpl_timeout = Int(30000)

# periodic_pay_core = And(Txn.type_enum() == Int(1),
#                         Txn.fee() < tmpl_fee)
                      
# periodic_pay_transfer = And(Txn.close_remainder_to() ==  Global.zero_address(),
#                             Txn.receiver() == tmpl_rcv,
#                             Txn.amount() == tmpl_amt,
#                             Txn.first_valid() % tmpl_period == Int(0),
#                             Txn.last_valid() == tmpl_dur + Txn.first_valid(),
#                             Txn.lease() == tmpl_x)

# periodic_pay_close = And(Txn.close_remainder_to() == tmpl_rcv,
#                          Txn.receiver() == Global.zero_address(),
#                          Txn.first_valid() > tmpl_timeout,
#                          Txn.amount() == Int(0))

# periodic_pay_escrow = And(periodic_pay_core, Or(periodic_pay_transfer, periodic_pay_close))

# print(periodic_pay_escrow.teal())

In [21]:
# This example is provided for informational purposes only and has not been audited for security.

from pyteal import *

"""Periodic Payment"""

tmpl_fee = Int(1000)
tmpl_period = Int(50)
tmpl_dur = Int(5000)
tmpl_lease = Bytes("base64", "023sdDE2")
tmpl_amt = Int(2000)
tmpl_rcv = Addr("6ZHGHH5Z5CTPCF5WCESXMGRSVK7QJETR63M3NY5FJCUYDHO57VTCMJOBGY")
tmpl_rcv = Addr("B767GZZJI4YEDKZU62INJGFIGSWQHDIH2HP6NRSSWFTFUYMCNU5LN2M53A")
tmpl_timeout = Int(30000)

def periodic_payment(tmpl_fee=tmpl_fee,
                     tmpl_period=tmpl_period,
                     tmpl_dur=tmpl_dur,
                     tmpl_lease=tmpl_lease,
                     tmpl_amt=tmpl_amt,
                     tmpl_rcv=tmpl_rcv,
                     tmpl_timeout=tmpl_timeout):

    periodic_pay_core = And(
        Txn.type_enum() == TxnType.Payment,
        Txn.fee() < tmpl_fee,
        Txn.first_valid() % tmpl_period == Int(0),
        Txn.last_valid() == tmpl_dur + Txn.first_valid(),
        Txn.lease() == tmpl_lease
    )

    periodic_pay_transfer = And(
        Txn.close_remainder_to() == Global.zero_address(),
        Txn.rekey_to() == Global.zero_address(),
        Txn.receiver() == tmpl_rcv,
        Txn.amount() == tmpl_amt
    )

    periodic_pay_close = And(
        Txn.close_remainder_to() == tmpl_rcv,
        Txn.rekey_to() == Global.zero_address(),
        Txn.receiver() == Global.zero_address(),
        Txn.first_valid() == tmpl_timeout,
        Txn.amount() == Int(0)
    )

    periodic_pay_escrow = periodic_pay_core.And(periodic_pay_transfer.Or(periodic_pay_close))

    return periodic_pay_escrow

if __name__ == "__main__":
#     use version 3
    print(compileTeal(periodic_payment(), mode=Mode.Signature, version=3))


#pragma version 3
txn TypeEnum
int pay
==
txn Fee
int 1000
<
&&
txn FirstValid
int 50
%
int 0
==
&&
txn LastValid
int 5000
txn FirstValid
+
==
&&
txn Lease
byte base64(023sdDE2)
==
&&
txn CloseRemainderTo
global ZeroAddress
==
txn RekeyTo
global ZeroAddress
==
&&
txn Receiver
addr B767GZZJI4YEDKZU62INJGFIGSWQHDIH2HP6NRSSWFTFUYMCNU5LN2M53A
==
&&
txn Amount
int 2000
==
&&
txn CloseRemainderTo
addr B767GZZJI4YEDKZU62INJGFIGSWQHDIH2HP6NRSSWFTFUYMCNU5LN2M53A
==
txn RekeyTo
global ZeroAddress
==
&&
txn Receiver
global ZeroAddress
==
&&
txn FirstValid
int 30000
==
&&
txn Amount
int 0
==
&&
||
&&


## Examples are available on 

## https://pyteal.readthedocs.io/en/stable/examples.html#periodic-payment

## or 

## https://github.com/algorand/pyteal/tree/master/examples



###### https://developer.algorand.org/docs/features/accounts/create/

###### goal kmd start -t 3600

In [23]:
#!/usr/bin/env python3

from pyteal import *
import uuid, params, base64
from algosdk.v2client import algod
from algosdk import transaction, account, mnemonic
import subprocess
# from periodic_payment import periodic_pay_escrow

#--------- compile & send transaction using Goal and Python SDK ----------
# periodic_pay_escrow = compileTeal(periodic_payment(), mode=Mode.Signature, version=3)
# teal_source = periodic_pay_escrow.teal() 
teal_source = compileTeal(periodic_payment(), mode=Mode.Signature, version=3)

# compile teal
teal_file = str(uuid.uuid4()) + ".teal"
with open(teal_file, "w+") as f:
    f.write(teal_source)
lsig_fname = str(uuid.uuid4()) + ".tealc"

# # stdout, stderr = execute(["goal", "clerk", "compile", "-o", lsig_fname,
# #                           teal_file])

# if stderr != "":
#     print(stderr)
#     raise
# elif len(stdout) < 59:
#     print("error in compile teal")
#     raise
subprocess.run(["goal", "clerk", "compile", "-o", lsig_fname,
                          teal_file])


with open(lsig_fname, "rb") as f:
    teal_bytes = f.read()
lsig = transaction.LogicSig(teal_bytes)

# create algod clients
acl = algod.AlgodClient(params.algod_token, params.algod_address)

#Recover the account that is wanting to delegate signature
passphrase = "patrol crawl rule faculty enemy sick reveal embody trumpet win shy zero ill draw swim excuse tongue under exact baby moral kite spring absent double"
sk = mnemonic.to_private_key(passphrase)
addr = account.address_from_private_key(sk)
print( "Address of Sender/Delgator: " + addr )

# sign the logic signature with an account sk
lsig.sign(sk)

# get suggested parameters
params = acl.suggested_params()
# gen = params["genesisID"]
# gh = params["genesishashb64"]
# startRound = params["lastRound"] - (params["lastRound"] % 1000)
# endRound = startRound + 1000
# fee = 1000
# amount = 2000

gen = params.gen
gh = params.gh
startRound = params.first
endRound = params.last
fee = params.min_fee
amount = 2000



receiver = "ZZAF5ARA4MEC5PVDOP64JM5O5MQST63Q2KOY2FLYFLXXD3PFSNJJBYAFZM"
lease = base64.b64decode("y9OJ5MRLCHQj8GqbikAUKMBI7hom+SOj8dlopNdNHXI=")

print(params.last)
print(startRound)
print(endRound)

# create a transaction
txn = transaction.PaymentTxn(addr, fee, startRound, endRound, gh, receiver, amount, flat_fee=True, lease=lease)

# Create the LogicSigTransaction with contract account LogicSig
lstx = transaction.LogicSigTransaction(txn, lsig)

# write to file
txns = [lstx]
transaction.write_to_file(txns, "p_pay.stxn")

# send raw LogicSigTransaction to network
txid = acl.send_transaction(lstx)
print("Transaction ID: " + txid)

Address of Sender/Delgator: B767GZZJI4YEDKZU62INJGFIGSWQHDIH2HP6NRSSWFTFUYMCNU5LN2M53A


AlgodHTTPError: {"message":"operation not available during catchup"}


## https://pyteal.readthedocs.io/en/stable/examples.html#periodic-payment

### Periodic Payment

###### Periodic Payment allows some account to execute periodic withdrawal of funds. This PyTeal program creates an contract account that allows tmpl_rcv to withdraw tmpl_amt every tmpl_period rounds for tmpl_dur after every multiple of tmpl_period.

###### After tmpl_timeout, all remaining funds in the escrow are available to tmpl_rcv.

In [33]:
# This example is provided for informational purposes only and has not been audited for security.

from pyteal import *

"""Periodic Payment"""

tmpl_fee = Int(1000)
tmpl_period = Int(50)
tmpl_dur = Int(5000)
tmpl_lease = Bytes("base64", "023sdDE2")
tmpl_amt = Int(2000)
tmpl_rcv = Addr("VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U")
tmpl_timeout = Int(30000)

def periodic_payment(tmpl_fee=tmpl_fee,
                     tmpl_period=tmpl_period,
                     tmpl_dur=tmpl_dur,
                     tmpl_lease=tmpl_lease,
                     tmpl_amt=tmpl_amt,
                     tmpl_rcv=tmpl_rcv,
                     tmpl_timeout=tmpl_timeout):

    periodic_pay_core = And(
        Txn.type_enum() == TxnType.Payment,
        Txn.fee() < tmpl_fee,
        Txn.first_valid() % tmpl_period == Int(0),
        Txn.last_valid() == tmpl_dur + Txn.first_valid(),
        Txn.lease() == tmpl_lease
    )

    periodic_pay_transfer = And(
        Txn.close_remainder_to() == Global.zero_address(),
        Txn.rekey_to() == Global.zero_address(),
        Txn.receiver() == tmpl_rcv,
        Txn.amount() == tmpl_amt
    )

    periodic_pay_close = And(
        Txn.close_remainder_to() == tmpl_rcv,
        Txn.rekey_to() == Global.zero_address(),
        Txn.receiver() == Global.zero_address(),
        Txn.first_valid() == tmpl_timeout,
        Txn.amount() == Int(0)
    )

    periodic_pay_escrow = periodic_pay_core.And(periodic_pay_transfer.Or(periodic_pay_close))

    return periodic_pay_escrow

if __name__ == "__main__":
    print(compileTeal(periodic_payment(), mode=Mode.Signature, version=3))
    #     write to a teal file
    with open('periodic_payment.teal', 'w') as f:
        compiled = compileTeal(periodic_payment(), mode=Mode.Signature, version=3)
        f.write(compiled)

#pragma version 3
txn TypeEnum
int pay
==
txn Fee
int 1000
<
&&
txn FirstValid
int 50
%
int 0
==
&&
txn LastValid
int 5000
txn FirstValid
+
==
&&
txn Lease
byte base64(023sdDE2)
==
&&
txn CloseRemainderTo
global ZeroAddress
==
txn RekeyTo
global ZeroAddress
==
&&
txn Receiver
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
==
&&
txn Amount
int 2000
==
&&
txn CloseRemainderTo
addr VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U
==
txn RekeyTo
global ZeroAddress
==
&&
txn Receiver
global ZeroAddress
==
&&
txn FirstValid
int 30000
==
&&
txn Amount
int 0
==
&&
||
&&


# TEAL




## Debug smart contract

#### https://developer.algorand.org/docs/features/asc1/debugging/


make sure node is running


run in cmd 
`tealdbg debug periodic_payment.teal`


open chrome and type

`chrome://inspect/`

for the first time you have to  click `Configure` to add `localhost:9392`

#### more on 
#### https://github.com/algorand/go-algorand/blob/master/cmd/tealdbg/README.md

!tealdbg debug periodic_payment.teal

In [24]:
def dryrun_debug(lstx, mysource):
    sources = []
    if (mysource != None):
        # source
        sources = [DryrunSource(field_name="lsig", source=mysource, txn_index=0)]
    drr = DryrunRequest(txns=[lstx], sources=sources)
    dryrun_response = algod_client.dryrun(drr)
    return dryrun_response

In [2]:
!goal node restart

The node was successfully stopped.
Algorand node successfully started!


## [Smart Contract in python v2 complete examples](https://github.com/algorand/docs/tree/master/examples/smart_contracts/v2/python)

In [3]:
import json
import base64

from algosdk.v2client import algod
from algosdk import account, mnemonic
from algosdk.v2client.models import DryrunRequest, DryrunSource
from algosdk.future.transaction import PaymentTxn, LogicSig, LogicSigTransaction
# from algosdk.testing import dryrun

def wait_for_confirmation(client, txid):
    """
    Utility function to wait until the transaction is
    confirmed before proceeding.
    """
    last_round = client.status().get('last-round')
    txinfo = client.pending_transaction_info(txid)
    while not (txinfo.get('confirmed-round') and txinfo.get('confirmed-round') > 0):
        print("Waiting for confirmation")
        last_round += 1
        client.status_after_block(last_round)
        txinfo = client.pending_transaction_info(txid)
    print("Transaction {} confirmed in round {}.".format(
        txid, txinfo.get('confirmed-round')))
    return txinfo

# dryrun source if provided, else dryrun compiled
def dryrun_debug(lstx, mysource):
    sources = []
    if (mysource != None):
        # source
        sources = [DryrunSource(field_name="lsig", source=mysource, txn_index=0)]
    drr = DryrunRequest(txns=[lstx], sources=sources)
    dryrun_response = algod_client.dryrun(drr)
    return dryrun_response

try:
    # Create an algod client
    algod_token = '25f90f9852eba8650750cb2e41b8dba807071d091168d9ece5554cabdaf5ce68'
    # algod_address = 'https://testnet-algorand.api.purestake.io/ps2'
    algod_address = 'http://localhost:8101'

    receiver = "VV3PUESFXXTZJBMLBOYN45WC56R4BJELVZSFLADYDYQKEQY7EPGSNQWT5U"
    algod_client = algod.AlgodClient(algod_token, algod_address)

    myprogram = "samplearg.teal"

    # Read TEAL program
    data = open(myprogram, 'r').read()

    # Compile TEAL program
    # // This code is meant for learning purposes only	
    # // It should not be used in production	
    # // samplearg.teal
    # arg_0	
    # btoi
    # int 123	
    # ==	

    # // bto1	
    # // Opcode: 0x17	
    # // Pops: ... stack, []byte	
    # // Pushes: uint64	
    # // converts bytes X as big endian to uint64	
    # // btoi panics if the input is longer than 8 bytes

    response = algod_client.compile(data)
    # Print(response)
    print("Response Result = ", response['result'])
    print("Response Hash = ", response['hash'])

    # Create logic sig
    programstr = response['result']
    t = programstr.encode()
    program = base64.decodebytes(t)
    print(program)
    
    # Create arg to pass
    # string parameter
    # arg_str = "<my string>"
    # arg1 = arg_str.encode()
    # lsig = transaction.LogicSig(program, args=[arg1])

    # integer parameter
    # arg1 = (123).to_bytes(8, 'big')
    # lsig = transaction.LogicSig(program, args=[arg1])
    # see more info here: https://developer.algorand.org/docs/features/asc1/sdks/#accessing-teal-program-from-sdks

    # if TEAL program requires an arg,
    # if not, omit args param on LogicSig
    # lsig = LogicSig(program)
    arg1 = (123).to_bytes(8, 'big')
    lsig = LogicSig(program, args=[arg1])

    # Recover the account that is wanting to delegate signature
    passphrase = "share tobacco purity drill pony lizard throw wild police foam street excuse ranch north shallow misery same cream piano useless sun cheap prevent able produce"
    sk = mnemonic.to_private_key(passphrase)
    addr = account.address_from_private_key(sk)
    print("Address of Sender/Delegator: " + addr)

    # Sign the logic signature with an account sk
    lsig.sign(sk)

    # Get suggested parameters
    params = algod_client.suggested_params()
    # Comment out the next two (2) lines to use suggested fees
    params.flat_fee = True
    params.fee = 1000

    # Build transaction
    amount = 10000
    closeremainderto = None

    # Create a transaction
    txn = PaymentTxn(
        addr, params, receiver, amount, closeremainderto)
    # Create the LogicSigTransaction with contract account LogicSig
    lstx = LogicSigTransaction(txn, lsig)
    # transaction.write_to_file([lstx], "./simple.stxn")

    # compile
    dryrun_response_compiled = dryrun_debug(lstx, None)
    print ("COMPILED Dryrun results...")
    print(json.dumps(dryrun_response_compiled, indent=2))
    
    # source   
#     dryrun_respone_source = dryrun_debug(lstx, source)
#     print("SOURCE Dryrun results...")
#     print(json.dumps(dryrun_respone_source, indent=2))
   
    # Send raw LogicSigTransaction to network

    txid = algod_client.send_transaction(lstx)
    print("Transaction ID: " + txid)
    wait_for_confirmation(algod_client, txid)
except Exception as e:
    print(e)

Response Result =  ASABey0XIhI=
Response Hash =  UVBYHRZIHUNUELDO6HWUAHOZF6G66W6T3JOXIIUSV3LDSBWVCFZ6LM6NCA
b'\x01 \x01{-\x17"\x12'
Address of Sender/Delegator: EWEFAZZF24OZK7SKZDCZHDIF73DKGRWOPFMPHJVYF3RPRIFZ4V32KFGBNU
COMPILED Dryrun results...
{
  "error": "",
  "protocol-version": "https://github.com/algorandfoundation/specs/tree/d050b3cade6d5c664df8bd729bf219f179812595",
  "txns": [
    {
      "disassembly": [
        "#pragma version 1",
        "intcblock 123",
        "arg_0",
        "btoi",
        "intc_0 // 123",
        "==",
        ""
      ],
      "logic-sig-messages": [
        "PASS"
      ],
      "logic-sig-trace": [
        {
          "line": 1,
          "pc": 1,
          "stack": []
        },
        {
          "line": 2,
          "pc": 4,
          "stack": []
        },
        {
          "line": 3,
          "pc": 5,
          "stack": [
            {
              "bytes": "AAAAAAAAAHs=",
              "type": 1,
              "uint": 0
            }


In [5]:
!goal app

Manage applications

Usage:
  goal app [flags]
  goal app [command]

Available Commands:
  call        Call an application
  clear       Clear out an application's state in your account
  closeout    Close out of an application
  create      Create an application
  delete      Delete an application
  info        Look up current parameters for an application
  interact    Interact with an application
  optin       Opt in to an application
  read        Read local or global state for an application
  update      Update an application's programs

Flags:
      --app-account strings        Accounts that may be accessed from application logic
      --app-arg strings            Args to encode for application transactions (all will be encoded to a byte slice). For ints, use the form 'int:1234'. For raw bytes, use the form 'b64:A=='. For printable strings, use the form 'str:hello'. For addresses, use the form 'addr:XYZ...'.
  -i, --app-input string           JSON file containing encoded argumen

In [9]:
!goal app call --app-id 1

required flag(s) "from" not set


Error: required flag(s) "from" not set
Usage:
  goal app call [flags]

Flags:
      --app-id uint                 Application ID
      --dryrun-dump                 Dump in dryrun format acceptable by dryrun REST api
      --dryrun-dump-format string   Dryrun dump format: json, msgp (default "json")
      --fee uint                    The transaction fee (automatically determined by default), in microAlgos
      --firstvalid uint             The first round where the transaction may be committed to the ledger
  -f, --from string                 Account to call app from
  -h, --help                        help for call
      --lastvalid uint              The last round where the transaction may be committed to the ledger
  -x, --lease string                Lease value (base64, optional): no transaction may also acquire this lease until lastvalid
  -N, --no-wait                     Don't wait for transaction to commit
  -n, --note string                 Note text (ignored if --noteb64 us

In [6]:
!goal account

Collection of commands to support the creation and management of accounts / wallets tied to a specific Algorand node instance.

Usage:
  goal account [flags]
  goal account [command]

Available Commands:
  addpartkey           Generate a participation key for the specified account
  balance              Retrieve the balances for the specified account
  changeonlinestatus   Change online status for the specified account
  delete               Delete an account
  dump                 Dump the balance record for the specified account
  export               Export an account key for use with account import
  import               Import an account key from mnemonic
  importrootkey        Import .rootkey files from the data directory into a kmd wallet
  info                 Retrieve information about the assets and applications belonging to the specified account
  installpartkey       Install a participation key
  list                 Show the list of Algorand accounts on this machine
  list

In [7]:
!goal account list

No wallets found. Create a new wallet with `goal wallet new [wallet name]`
